In [1]:
from pathlib import Path
from chic import Structure

## Coarse-graining basics (ZIF-8 example)

In this example we take the CIF for the prototypical zeolitic imidazolate 
framework, ZIF-8, and coarse-grain it. We check the coarse-graining is 
reasonable by overlaying the atomistic structure and coarse-grained structure 
before writing the final net in TopoCIF format.

In [2]:
# path to test example directory.
eg_dir = Path('cg_zif8')

# instantiate a Structure object from a CIF file.
zif8 = Structure.from_cif(eg_dir / 'ZIF-8-sod.cif', cores=4, verbose=True)

/Users/tcnicholas/miniconda3/envs/chic23/lib/python3.8/site-packages/chic/cif.py:107: UserWarning: Unable to match CIF atoms to Pymatgen atoms.
  warnings.warn('Unable to match CIF atoms to Pymatgen atoms.')


In [3]:
# we can apply methods native to the Pymatgen Structure class. here we remove 
# all oxygen atoms from the structure, which reside in the pores.
zif8.remove_species('O')

In [4]:
# we can apply chic methods for dealing with structural distorder. here we
# take the average position between pairs of H atoms.
zif8.average_element_pairs('H', rmin=0.0, rmax=0.9)

average_element_pairs() took 0.02 seconds to execute.


In [5]:
# compute the neighbour list using the CrystalNN algorithm.
zif8.get_neighbours_crystalnn()

# determine atomic clusters.
zif8.find_atomic_clusters()

# coarse-grain with centroid method.
zif8.get_coarse_grained_net()

get_neighbours_crystalnn() took 9.10 seconds to execute.
find_atomic_clusters() took 0.03 seconds to execute.
get_coarse_grained_net() took 0.01 seconds to execute.


In [6]:
# we can write the coarse-grained net with the bond information to a TopoCIF.
zif8.net_to_cif(eg_dir / 'ZIF-8-sod-cg.cif', write_bonds=True, name='ZIF-8-cg')

# we can also write to LAMMPS data file.
zif8.net_to_lammps_data(
    eg_dir / 'ZIF-8-sod-cg.data', write_bonds=True, name='ZIF-8-cg'
)

# we can check the coarse-grained net seems reasonable.
zif8.overlay_cg_atomistic_representations(eg_dir / 'ZIF-8-sod-overlay.cif')

In [7]:
# call Pymatgen's writer to write to a VASP POSCAR format.
zif8.net_to(eg_dir / 'POSCAR', fmt='poscar')

In [8]:
# use ASE writer to write an xyz file.
zif8.net_to_ase_to(eg_dir / 'ZIF-8-cg.xyz')

In [9]:
# net to Pymatgen Structure object.
pym_net = zif8.net_to_struct()
type(pym_net), pym_net

(pymatgen.core.structure.Structure,
 Structure Summary
 Lattice
     abc : 16.8509 16.8509 16.8509
  angles : 90.0 90.0 90.0
  volume : 4784.860756696228
       A : 16.8509 0.0 1.0318200373876067e-15
       B : -1.0318200373876067e-15 16.8509 1.0318200373876067e-15
       C : 0.0 0.0 16.8509
     pbc : True True True
 PeriodicSite: O (6.252, 6.252, 0.2886) [0.371, 0.371, 0.01713]
 PeriodicSite: O (0.2886, 10.6, 10.6) [0.01713, 0.629, 0.629]
 PeriodicSite: O (0.2886, 6.252, 6.252) [0.01713, 0.371, 0.371]
 PeriodicSite: O (10.6, 10.6, 0.2886) [0.629, 0.629, 0.01713]
 PeriodicSite: O (10.6, 0.2886, 10.6) [0.629, 0.01713, 0.629]
 PeriodicSite: O (6.252, 0.2886, 6.252) [0.371, 0.01713, 0.371]
 PeriodicSite: O (8.137, 14.68, 2.173) [0.4829, 0.871, 0.129]
 PeriodicSite: O (6.252, 16.56, 10.6) [0.371, 0.9829, 0.629]
 PeriodicSite: Si (8.425, 12.64, 1.29e-15) [0.5, 0.75, 0.0]
 PeriodicSite: O (8.714, 14.68, 14.68) [0.5171, 0.871, 0.871]
 PeriodicSite: Si (8.425, 0.0, 12.64) [0.5, 0.0, 0.75]
 Pe

In [10]:
# net to ASE Atoms object.
ase_net = zif8.net_to_ase_atoms()
type(ase_net), ase_net

(ase.atoms.Atoms,
 Atoms(symbols='O24Si12', pbc=True, cell=[[16.8509, 0.0, 1.0318200373876067e-15], [-1.0318200373876067e-15, 16.8509, 1.0318200373876067e-15], [0.0, 0.0, 16.8509]]))